# Reservoir for Sequential Data

In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
class EchoStateNetwork:
    def __init__(self, input_dim, reservoir_size, output_dim, spectral_radius=0.9, alpha=1e-6):
        self.input_dim = input_dim
        self.reservoir_size = reservoir_size
        self.output_dim = output_dim
        self.spectral_radius = spectral_radius
        self.alpha = alpha
        self.initialize_weights()
        self.readout = Ridge(alpha=self.alpha)

    def initialize_weights(self):
        self.Win = torch.rand((self.reservoir_size, self.input_dim)) * 2 - 1
        self.W = torch.rand((self.reservoir_size, self.reservoir_size)) * 2 - 1
        self.adjust_spectral_radius()
        
    def adjust_spectral_radius(self):
        eigenvalues, _ = torch.linalg.eig(self.W)
        rhoW = max(abs(eigenvalues))
        self.W *= self.spectral_radius / rhoW

    def train(self, train_data, target_data):
        state = torch.zeros(self.reservoir_size)
        states = []
        
        for t in range(len(train_data)):
            u = torch.tensor([train_data[t]], dtype=torch.float32)
            state = torch.tanh(self.Win @ u + self.W @ state)
            states.append(state.numpy())
        
        states = np.array(states)
        self.readout.fit(states, target_data)
        predictions = self.readout.predict(states)
        return predictions

def train_esn(model, train_data, target_data, test_data, test_target, num_trials=50):
    results = []
    test_results = []
    for trial in range(num_trials):
        torch.manual_seed(trial)
        np.random.seed(trial)  # Ensure NumPy is also seeded
        model.initialize_weights()

        predictions = model.train(train_data, target_data)
        mse = mean_squared_error(target_data, predictions)
        results.append(mse)
    
        # Evaluate on test set
        test_predictions = model.train(test_data, test_target)
        test_mse = mean_squared_error(test_target, test_predictions)
        test_results.append(test_mse)
        
        print(f'Trial {trial + 1}/{num_trials}: Train MSE = {mse}, Test MSE = {test_mse}')
    return results, test_results

In [3]:
def baseline_model(train_data, target_data, test_data, test_target):
    model = LinearRegression()
    model.fit(train_data.reshape(-1, 1), target_data)
    train_predictions = model.predict(train_data.reshape(-1, 1))
    test_predictions = model.predict(test_data.reshape(-1, 1))
    train_mse = mean_squared_error(target_data, train_predictions)
    test_mse = mean_squared_error(test_target, test_predictions)
    return train_mse, test_mse

In [5]:
def mackey_glass(T, tau=17, n=10, beta=0.2, gamma=0.1):
    x = np.zeros(T)
    x[0] = 1.2
    for t in range(1, T):
        if t - tau >= 0:
            x[t] = x[t-1] + beta * x[t-tau] / (1 + x[t-tau]**n) - gamma * x[t-1]
        else:
            x[t] = x[t-1]
    return x

In [6]:
# Generate Mackey-Glass time series data
T = 1500
data = mackey_glass(T)
train_data, train_target = data[:800], data[200:1000]
test_data, test_target = data[800:1300], data[1000:1500]

neurons = 100  # Number of neurons in the reservoir
num_trials = 50  # Number of trials with different initializations
input_dim = 1
output_dim = 1
esn = EchoStateNetwork(input_dim, neurons, output_dim)

train_results, test_results = train_esn(esn, train_data, train_target, test_data, test_target, num_trials)
print(f'Average Train MSE over {num_trials} trials: {np.mean(train_results)}')
print(f'Average Test MSE over {num_trials} trials: {np.mean(test_results)}')


Trial 1/50: Train MSE = 0.014141706764518256, Test MSE = 0.02583314880938891
Trial 2/50: Train MSE = 0.013283701805913673, Test MSE = 0.023183119807389796
Trial 3/50: Train MSE = 0.013635773117935629, Test MSE = 0.024390907586079428
Trial 4/50: Train MSE = 0.013601536545094457, Test MSE = 0.023273827424109498
Trial 5/50: Train MSE = 0.013505257087724144, Test MSE = 0.023914189015117427
Trial 6/50: Train MSE = 0.013938276885673872, Test MSE = 0.02574170856875841
Trial 7/50: Train MSE = 0.01390492908456615, Test MSE = 0.025489316191268735
Trial 8/50: Train MSE = 0.013587816414938954, Test MSE = 0.024172636906397444
Trial 9/50: Train MSE = 0.013705599141463763, Test MSE = 0.024578962054267114
Trial 10/50: Train MSE = 0.013817790392645102, Test MSE = 0.02526466227624872
Trial 11/50: Train MSE = 0.013884169989002099, Test MSE = 0.02546055336945055
Trial 12/50: Train MSE = 0.014245300759134185, Test MSE = 0.028112336214827425
Trial 13/50: Train MSE = 0.013697304834924772, Test MSE = 0.024762

In [7]:
# Baseline model performance
baseline_train_mse, baseline_test_mse = baseline_model(train_data, train_target, test_data, test_target)
print(f'Baseline Train MSE: {baseline_train_mse}')
print(f'Baseline Test MSE: {baseline_test_mse}')

Baseline Train MSE: 0.026879490465194618
Baseline Test MSE: 0.04844089867569214
